In [1]:
%%capture
import warnings

# Suppress specific Intel MKL warnings
warnings.filterwarnings("ignore")#, category=DeprecationWarning, module="mkl")


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime

from itertools import combinations
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA as ARIMA
import statsmodels.api as sm
import statsmodels.tsa.api as smt
pd.options.display.float_format = '{:.2f}'.format

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [8]:
client = pd.read_csv("../../data/client.csv")
ePrices = pd.read_csv("../../data/electricity_prices.csv")
gasPrices = pd.read_csv("../../data/gas_prices.csv")
train = pd.read_csv("../../data/train.csv")

weatherPredInt = pd.read_csv("../model_architecture_study/interpolPredWeather.csv")
weatherHistInt = pd.read_csv("../histWeatherSnowCover.csv")

# data processing

In [9]:
producing = train.loc[train.is_consumption == 0]
consuming = train.loc[train.is_consumption == 1]
train = pd.merge(producing.drop('is_consumption',axis = 1), consuming.drop('is_consumption',axis = 1),on=['data_block_id','prediction_unit_id','datetime','county','is_business','product_type'], how='outer',suffixes=('_prod', '_cons'))
del producing, consuming
print(train.shape)

clientsTime = pd.merge(train, client, on=['county','is_business','product_type','data_block_id'], how='inner')

clientsTime['datetime'] = pd.to_datetime(clientsTime['datetime'])
clientsTime['yearday'] = clientsTime['datetime'].dt.day_of_year
clientsTime['weekday'] = clientsTime['datetime'].dt.day_of_week
clientsTime['month'] = clientsTime['datetime'].dt.month
clientsTime['monthday'] = clientsTime['datetime'].dt.day
clientsTime['year'] = clientsTime['datetime'].dt.year
clientsTime['hour'] = clientsTime['datetime'].dt.hour

unique_pairs = list(set(zip(clientsTime['is_business'], clientsTime[ 'product_type'])))
pair_index_dict = {pair: index for index, pair in enumerate(unique_pairs)}
clientsTime['business_prodType'] = list(map(pair_index_dict.get, zip(clientsTime['is_business'], clientsTime['product_type'])))

unique_pairs_cust = list(set(zip(clientsTime['is_business'], clientsTime[ 'product_type'], clientsTime['county'], clientsTime['eic_count'],clientsTime['installed_capacity'])))
pair_index_dict = {pair: index for index, pair in enumerate(unique_pairs_cust)}
clientsTime['ind_customer_id'] = list(map(pair_index_dict.get, zip(clientsTime['is_business'], clientsTime['product_type'], clientsTime['county'], clientsTime['eic_count'],clientsTime['installed_capacity'])))

# interpolate daylight savings
clientsTime = clientsTime.interpolate()


import holidays
from datetime import date

us_holidays = holidays.EE()  # this is a dict

clientsTime['holiday'] = clientsTime['datetime'].apply(lambda s : s in us_holidays)
clientsTime['no_workday'] = ((clientsTime['holiday']) | (clientsTime['weekday'] > 4))

ePrices['forecast_date'] = pd.to_datetime(ePrices['forecast_date'])
ePrices['hour'] = ePrices.forecast_date.dt.hour
# Set 'timestamp' as the index
ePrices.set_index('forecast_date', inplace=True)

# Resample to fill missing hours
df_resampled = ePrices.resample('1H').asfreq()

# Linearly interpolate missing values
ePrices = df_resampled.interpolate(method='linear')

ePrices['euros_per_mwh'] = ePrices.euros_per_mwh.replace(4000.0, np.nan)

# Resample to fill missing hours
df_resampled = ePrices.resample('1H').asfreq()

# Linearly interpolate missing values
ePrices = df_resampled.interpolate(method='linear')


clientsTime2 = clientsTime.set_index(['datetime','prediction_unit_id'])
clientsTime2 = pd.merge(clientsTime,ePrices[['data_block_id','hour','euros_per_mwh']], on=['data_block_id','hour'])#.set_index(['datetime','prediction_unit_id'])
clientsTime2 = pd.merge(clientsTime2, gasPrices[['lowest_price_per_mwh','highest_price_per_mwh','data_block_id']], on=['data_block_id'])

weatherPredInt['hour'] = weatherPredInt['hours_ahead'] - 24
weatherPredInt['county'] = weatherPredInt['County']
weatherPredInt.drop('County',axis = 1, inplace=True)
weatherPredInt.drop('Unnamed: 0', axis = 1, inplace=True)
weatherPredInt.set_index(['forecast_datetime','county'])
clientsTime3 = pd.merge(clientsTime2, weatherPredInt, on=['county', 'hour','data_block_id'], how='inner', suffixes=('','_pred'))

weatherHistInt['datetime'] = pd.to_datetime(weatherHistInt['datetime'])
weatherHistInt['hour'] = weatherHistInt.datetime.dt.hour
weatherHistInt['county'] = weatherHistInt['County']
weatherHistInt.drop(['Unnamed: 0.1', 'Unnamed: 0','County'], axis = 1, inplace=True)

#move hour timestamp 11 hours up and deal with proper time separation
weatherHistInt['hour_original'] = weatherHistInt.datetime.dt.hour
weatherHistInt['hour'] = weatherHistInt.datetime.dt.hour-11
weatherHistInt['hour'] = weatherHistInt['hour'].apply(lambda s : s+24 if s<0 else s)

weatherHistInt.loc[weatherHistInt.data_block_id == 3].hour.unique()
mergedData = pd.merge(clientsTime3, weatherHistInt, on=['county', 'data_block_id', 'hour'], suffixes=('','_hist'))

mergedData = mergedData.set_index(['ind_customer_id','datetime']).sort_index()

(1009176, 10)


In [15]:
mergedData['old_target_prod'] = mergedData.groupby(['prediction_unit_id'])[['target_prod']].shift(48)
mergedData['old_target_cons'] = mergedData.groupby(['prediction_unit_id'])[['target_cons']].shift(48)

# 